## Helixscope Developer Queries

This notebook explains the metadata and back-end queries required to build the Helixscope project. 

Note ISO2 codes are used to link the `gadm28_countries` table to the summary data tables. The below mapping links the codes to human-readable names for the front-end.


*Specific warming levels*
```json
{'1.5':'1.5°C specific warming level',
 '2':'2°C specific warming level',
 '4':'4°C specific warming level'}
```


*Climate Variables*

```json
{'time_perc_change_SPI48':'info',
'nbp':'info',
'time_perc_change_SPI6':'info',
'time_perc_change_SRI48':'info',
'gpp':'info',
'evap':'info',
'perc_change_low_roff':'info',
'perc_change_roff':'info',
'time_perc_change_SRI6':'info'}

```


*ISO2 codes*
```json
{'AL': 'Albania',
 'AT': 'Austria',
 'BA': 'Bosnia and Herzegovina',
 'BE': 'Belgium',
 'BG': 'Bulgaria',
 'CH': 'Switzerland',
 'CY': 'Cyprus',
 'CZ': 'Czech Republic',
 'DE': 'Germany',
 'DK': 'Denmark',
 'EE': 'Estonia',
 'ES': 'Spain',
 'FI': 'Finland',
 'FR': 'France',
 'GB': 'United Kingdom',
 'GF': 'French Guiana',
 'GR': 'Greece',
 'HR': 'Croatia',
 'HU': 'Hungary',
 'IE': 'Ireland',
 'IL': 'Israel',
 'IS': 'Iceland',
 'IT': 'Italy',
 'LI': 'Liechtenstein',
 'LT': 'Lithuania',
 'LU': 'Luxembourg',
 'LV': 'Latvia',
 'MK': 'Macedonia',
 'MT': 'Malta',
 'NL': 'Netherlands',
 'NO': 'Norway',
 'PL': 'Poland',
 'PS': 'Palestina',
 'PT': 'Portugal',
 'RE': 'Reunion',
 'RO': 'Romania',
 'RS': 'Serbia',
 'SE': 'Sweden',
 'SI': 'Slovenia',
 'SK': 'Slovakia',
 'TR': 'Turkey'}
```


In [1]:
import folium
import pprint
import requests
import json

## Generate Choropleths 

Country-wide choropleths based on means.

In [10]:
swl_variable = '1.5'
climate_variable = 'cSoil'
data_table = 'joined_summaries_2017_08_27'

query = """WITH data as (SELECT * FROM joined_summaries_2017_08_27
 Where variable like 'cSoil'
 AND swl_info = '1.5')
 SELECT gadm28_countries.iso2, gadm28_countries.the_geom_webmercator,
 gadm28_countries.cartodb_id, mean as mean from data inner join gadm28_countries 
 on data.iso2=gadm28_countries.iso2
"""

style = """#layer {polygon-fill: ramp([mean], colorbrewer(Reds), jenks());
                   polygon-opacity: 0.9;}"""

account = 'helixscope'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}

r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.url)
pprint.pprint(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm2 = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm2)

map_osm2

https://helixscope.carto.com/api/v1/map
{'cdn_url': {'http': 'ashbu.cartocdn.com',
             'https': 'cartocdn-ashbu.global.ssl.fastly.net',
             'templates': {'http': {'subdomains': ['0', '1', '2', '3'],
                                    'url': 'http://{s}.ashbu.cartocdn.com'},
                           'https': {'subdomains': ['a', 'b', 'c', 'd'],
                                     'url': 'https://cartocdn-ashbu_{s}.global.ssl.fastly.net'}}},
 'last_updated': '2017-08-28T08:46:02.798Z',
 'layergroupid': 'f80e86cf1e517683b3ac17872dccdb84:1503909962798',
 'metadata': {'analyses': [],
              'dataviews': {},
              'layers': [{'id': 'layer0',
                          'meta': {'cartocss': '#layer {polygon-fill: '
                                               '#fee5d9;[ mean > '
                                               '3.12408299981965 ] '
                                               '{polygon-fill: #fcae91;}[ mean '
                              

## Example of retrieving data JSON

This will be necessary to build widgets attached to the data.

In [14]:
# example of getting the raw data

swl_variable = '1.5'
climate_variable = 'cSoil'
data_table = 'joined_summaries_2017_08_27'

query = """SELECT iso2, mean, max, count, min, std, model_long_name, model_short_name, model_taxonomy
 FROM joined_summaries_2017_08_27
 Where variable like 'cSoil'
 AND swl_info = '1.5'
"""

account = 'helixscope'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": query}
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://helixscope.carto.com/api/v2/sql?q=SELECT+iso2%2C+mean%2C+max%2C+count%2C+min%2C+std%2C+model_long_name%2C+model_short_name%2C+model_taxonomy%0A+FROM+joined_summaries_2017_08_27%0A+Where+variable+like+%27cSoil%27%0A+AND+swl_info+%3D+%271.5%27%0A
{'fields': {'count': {'type': 'number'},
            'iso2': {'type': 'string'},
            'max': {'type': 'number'},
            'mean': {'type': 'number'},
            'min': {'type': 'number'},
            'model_long_name': {'type': 'string'},
            'model_short_name': {'type': 'string'},
            'model_taxonomy': {'type': 'string'},
            'std': {'type': 'number'}},
 'rows': [{'count': 307,
           'iso2': 'NO',
           'max': 21.6948356628418,
           'mean': 13.4384078420335,
           'min': 10.1957817077637,
           'model_long_name': 'ORCHIDEE',
           'model_short_name': 'orchidee',
           'model_taxonomy': 'orchidee-giss-ecearth',
           'std': 2.05501450545896},
          {'count': 